In [1]:
# Define area of interest
from rippl.orbit_geometry.read_write_shapes import ReadWriteShapes
Benelux_shape = [[7.218017578125001, 53.27178347923819],
                 [7.00927734375, 53.45534913802113],
                 [6.932373046875, 53.72921671251272],
                 [6.756591796875, 53.68369534495075],
                 [6.1962890625, 53.57293832648609],
                 [5.218505859375, 53.50111704294316],
                 [4.713134765624999, 53.20603255157844],
                 [4.5703125, 52.80940281068805],
                 [4.2626953125, 52.288322586002984],
                 [3.856201171875, 51.88327296443745],
                 [3.3508300781249996, 51.60437164681676],
                 [3.284912109375, 51.41291212935532],
                 [2.39501953125, 51.103521942404186],
                 [2.515869140625, 50.78510168548186],
                 [3.18603515625, 50.5064398321055],
                 [3.8452148437499996, 50.127621728300475],
                 [4.493408203125, 49.809631563563094],
                 [5.361328125, 49.475263243037986],
                 [6.35009765625, 49.36806633482156],
                 [6.602783203124999, 49.6462914122132],
                 [6.536865234375, 49.83798245308484],
                 [6.251220703125, 50.085344397538876],
                 [6.448974609375, 50.42251884281916],
                 [6.218261718749999, 50.75035931136963],
                 [6.13037109375, 51.034485632974125],
                 [6.2841796875, 51.32374658474385],
                 [6.218261718749999, 51.59754765771458],
                 [6.2841796875, 51.754240074033525],
                 [6.767578125, 51.896833883012484],
                 [7.086181640625, 52.17393169256849],
                 [7.0751953125, 52.482780222078226],
                 [6.844482421875, 52.482780222078226],
                 [6.83349609375, 52.5897007687178],
                 [7.0751953125, 52.6030475337285],
                 [7.218017578125001, 53.27178347923819]]
study_area = ReadWriteShapes()
study_area(Benelux_shape)
study_area_shape = study_area.shape.buffer(0.2)

In [2]:
import datetime
from rippl.processing_templates.NWP_template import APSTemplate

# Track and data type of Sentinel data
mode = 'IW'
product_type = 'SLC'
polarisation = ['VV']

# Create the list of the 4 different stacks.
track_no = 37
stack_name = 'Benelux_NWP_track_37'
no_processes = 6

# For every track we have to select a master date. This is based on the search results earlier.
# Choose the date with the lowest coverage to create an image with only the overlapping parts.
master_date = datetime.datetime(year=2017, month=7, day=24)
start_date = datetime.datetime(year=2017, month=7, day=16)
end_date = datetime.datetime(year=2017, month=7, day=28)
aps_processing = APSTemplate(processes=no_processes)
aps_processing.download_sentinel_data(start_date=start_date, end_date=end_date, track=track_no,
                                           polarisation=polarisation, shapefile=study_area.shape, data=True, source='ASF')
aps_processing.create_sentinel_stack(start_date=start_date, end_date=end_date, master_date=master_date, cores=no_processes,
                                          track=track_no,stack_name=stack_name, polarisation=polarisation,
                                          shapefile=study_area.shape, mode=mode, product_type=product_type)

# Finally load the stack itself. If you want to skip the download step later, run this line before other steps!
aps_processing.read_stack(start_date=start_date, end_date=end_date, stack_name=stack_name)

searching for new POE files
https://scihub.copernicus.eu/gnss/search/?q=producttype:AUX_POEORB platformname:Sentinel-1 beginPosition:[2017-07-16T00:00:00Z TO 2017-07-28T00:00:00Z] endPosition:[2017-07-16T00:00:00Z TO 2017-07-28T00:00:00Z]&format=json
found 20 OSV results
/mnt/external/orbit_database/Sentinel-1/precise/S1B_OPER_AUX_POEORB_OPOD_20210308T210223_V20170722T225942_20170724T005942.EOF
/mnt/external/orbit_database/Sentinel-1/precise/S1B_OPER_AUX_POEORB_OPOD_20210308T203847_V20170721T225942_20170723T005942.EOF
/mnt/external/orbit_database/Sentinel-1/precise/S1B_OPER_AUX_POEORB_OPOD_20210308T221527_V20170725T225942_20170727T005942.EOF
/mnt/external/orbit_database/Sentinel-1/precise/S1B_OPER_AUX_POEORB_OPOD_20210308T212651_V20170723T225942_20170725T005942.EOF
/mnt/external/orbit_database/Sentinel-1/precise/S1B_OPER_AUX_POEORB_OPOD_20210308T215058_V20170724T225942_20170726T005942.EOF
/mnt/external/orbit_database/Sentinel-1/precise/S1B_OPER_AUX_POEORB_OPOD_20210308T183739_V20170716

In [3]:
# Processing of basic stack (See other tutorial for detailed information)
# Some basic settings for DEM creation.
dem_buffer = 0.1        # Buffer around radar image where DEM data is downloaded
dem_rounding = 0.1      # Rounding of DEM size in degrees
dem_type = 'SRTM3'      # DEM type of data we download (SRTM1, SRTM3 and TanDEM-X are supported)

# Define both the coordinate system of the full radar image and imported DEM
aps_processing.create_radar_coordinates()
aps_processing.create_dem_coordinates(dem_type=dem_type)

# Download external DEM
aps_processing.download_external_dem(buffer=dem_buffer, rounding=dem_rounding, n_processes=no_processes, dem_type=dem_type)

# Geocoding of image
aps_processing.geocoding()

# Next step applies resampling and phase correction in one step.
# Polarisation
aps_processing.geometric_coregistration_resampling(polarisation=polarisation, output_phase_correction=True)

# Resolution of output georeferenced grid
dy = 250
dx = 250

# The actual creation of the calibrated amplitude images
aps_processing.create_ml_coordinates(standard_type='oblique_mercator', dx=dx, dy=dy, buffer=0, rounding=0)
aps_processing.prepare_multilooking_grid(polarisation)
aps_processing.create_calibrated_amplitude_multilooked(polarisation)
aps_processing.create_output_tiffs_amplitude()

aps_processing.create_ifg_network(temporal_baseline=30)
aps_processing.create_interferogram_multilooked(polarisation)
aps_processing.create_coherence_multilooked(polarisation)

# Create output geotiffs
aps_processing.create_output_tiffs_coherence_ifg()

# Create lat/lon/incidence angle/DEM for multilooked grid.
aps_processing.create_geometry_mulitlooked()
aps_processing.create_output_tiffs_geometry()

All needed SRTM DEM files already downloaded.
Processing pipeline block 1 out of 3
Start time block 1 is 2021-06-14 17:24:04.934558
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already fin

/home/gert/Software/rippl_main/rippl/meta_data/image_data.py:564: RuntimeWarning: divide by zero encountered in log
  data.GetRasterBand(1).WriteArray(np.log(np.abs(self.memory['data'])))


Processing pipeline block 1 out of 1
Start time block 1 is 2021-06-14 17:27:25.200666
Skipping processing. Process already finished
Finished processing pipeline block 1 out of 1
Finished block 1 in 0:00:01.802679
Processing pipeline block 1 out of 1
Start time block 1 is 2021-06-14 17:27:28.950076
Skipping processing. Process already finished
Finished processing pipeline block 1 out of 1
Finished block 1 in 0:00:00.208464
Processing pipeline block 1 out of 1
Start time block 1 is 2021-06-14 17:27:30.282547
Skipping processing. Process already finished
Finished processing pipeline block 1 out of 1
Finished block 1 in 0:00:00.266961


In [4]:
# Unwrapping
aps_processing.create_unwrapped_images(polarisation=polarisation)
aps_processing.create_output_tiffs_unwrap()

Processing pipeline block 1 out of 1
Start time block 1 is 2021-06-14 17:27:32.150741
Skipping processing. Process already finished
Finished processing pipeline block 1 out of 1
Finished block 1 in 0:00:00.240095


In [5]:
# Create basic geometry for APS calculations.
# Create DEM and geometry for the APS interpolation grid
aps_processing.create_aps_coordinates(standard_type='oblique_mercator', dx=2500, dy=2500, buffer=1, rounding=0.1)
aps_processing.create_aps_geometry()
aps_processing.create_output_tiffs_aps_geometry()

Processing pipeline block 1 out of 1
Start time block 1 is 2021-06-14 17:27:36.653355
Skipping processing. Process already finished
Finished processing pipeline block 1 out of 1
Finished block 1 in 0:00:02.074581
Processing pipeline block 1 out of 1
Start time block 1 is 2021-06-14 17:27:41.027122
Skipping processing. Process already finished
Finished processing pipeline block 1 out of 1
Finished block 1 in 0:00:00.232041
Processing pipeline block 1 out of 1
Start time block 1 is 2021-06-14 17:27:43.549727
Skipping processing. Process already finished
Finished processing pipeline block 1 out of 1
Finished block 1 in 0:00:00.192873


In [6]:
aps_processing.create_plot_dem()
aps_processing.create_plot_incidence_angle()
aps_processing.create_plots_coherence()
aps_processing.create_plots_ifg()
aps_processing.create_plots_looks()
aps_processing.create_plots_unwrapped(overwrite=True)
aps_processing.create_plots_amplitude()

/home/gert/anaconda3/envs/stereoid/lib/python3.7/site-packages/matplotlib/colors.py:527: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/home/gert/anaconda3/envs/stereoid/lib/python3.7/site-packages/matplotlib/colors.py:527: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


The following steps will use Harmonie data that was provided by the KNMI (knmi.nl), the Royal Dutch Meteorological Institute
This data is added to the paper to run this tutorial, but is not directly downloadable from their databases. Data is
available upon request with a processing fee. For other national forecast agencies within europe other rules could apply.

In [7]:
import os
from rippl.user_settings import UserSettings
import urllib.request

settings = UserSettings()
settings.load_settings()

# Download of Harmonie data. This data is stored with the paper itself (should go relatively fast)
urls = ['https://surfdrive.surf.nl/files/index.php/s/kwXjZoRqujHRjsC/download',
        'https://surfdrive.surf.nl/files/index.php/s/N8RCpbFZjPrY1ME/download']
filenames = ['HA38_N25_201707180300_00245_GB',
             'HA38_N25_201707240300_00245_GB']
download_folder = os.path.join(settings.NWP_model_database, 'harmonie_data', 'h38')
if not os.path.exists(download_folder):
    os.mkdirs(download_folder)

for url, filename in zip(urls, filenames):
    if not os.path.exists(os.path.join(download_folder, filename)):
        urllib.request.urlretrieve(url, os.path.join(download_folder, filename))

In [8]:
# The first step we take is calculate the images for the two dates
# Then do the APS calculations
aps_processing.calculate_harmonie_aps(correct_time_delay=False)

# Then we create the interferogram of both images
aps_processing.calculate_ifg_harmonie_aps()

Processing pipeline block 1 out of 1
Start time block 1 is 2021-06-14 17:30:31.823052
Skipping processing. Process already finished
Skipping processing. Process already finished
Finished processing pipeline block 1 out of 1
Finished block 1 in 0:00:00.283472
Processing pipeline block 1 out of 1
Start time block 1 is 2021-06-14 17:30:33.641016
Skipping processing. Process already finished
Finished processing pipeline block 1 out of 1
Finished block 1 in 0:00:00.263722


In [9]:
# Create image and .tiff file for ECMWF individual dates and ifg
aps_processing.create_output_tiffs_harmonie_aps()
aps_processing.create_plots_harmonie_aps()

The following two steps make use of the ERA5 model from ECMWF.

In [10]:
from rippl.NWP_model_delay.load_NWP_data.ecmwf.ecmwf_type import ECMWFType
from rippl.NWP_model_delay.download_NWP_data.ecmwf.ecmwf_download import ECMWFdownload
import numpy as np
import os
from rippl.user_settings import UserSettings

settings = UserSettings()
settings.load_settings()
# Download of ECMWF data. (This can take some time because you have to wait in the queue!)
# Make sure you created your own cdsapi token!
# Find the guidelines for downloading via https://cds.climate.copernicus.eu/api-how-to

# We project_functions data for western europe. (This can be extended or narrowed, but should at least include the satellite
# orbit and radar points on the ground.)
latlim = [45, 56]
lonlim = [-2, 12]

data_type = 'era5'
time_interp = 'nearest'

# Take only part of dataset
overpasses = np.array([start_date, end_date])

ecmwf_type = ECMWFType(data_type)
download_folder = os.path.join(settings.NWP_model_database, 'ecmwf_data')
if not os.path.exists(download_folder):
    os.mkdirs(download_folder)

down = ECMWFdownload(latlim, lonlim, download_folder, data_type=data_type)
down.prepare_download(overpasses)
down.download()

In [11]:
# Create image for ECMWF

# The first step we take is calculate the images for the two dates
# Then do the APS calculations
aps_processing.calculate_ecmwf_aps(latlim=latlim, lonlim=lonlim)

# Then we create the interferogram of both images
aps_processing.calculate_ifg_ecmwf_aps()

Processing pipeline block 1 out of 1
Start time block 1 is 2021-06-14 17:31:27.794204
Skipping processing. Process already finished
Skipping processing. Process already finished
Finished processing pipeline block 1 out of 1
Finished block 1 in 0:00:00.265487
Processing pipeline block 1 out of 1
Start time block 1 is 2021-06-14 17:31:29.527881
Skipping processing. Process already finished
Finished processing pipeline block 1 out of 1
Finished block 1 in 0:00:00.214704


In [12]:
# Create image and .tiff file for ECMWF individual dates and ifg
aps_processing.create_output_tiffs_ecmwf_aps()
aps_processing.create_plots_ecmwf_aps()
